<h1> Cardernos de IA - Aprendizagem de Máquina </h1>
<h2> Regras de Associação - Parte 2</h2>
<h3> by Ronaldo F. Ramos </h3>

Neste caderno damos continuidade ao estudo de regras de associação. Vimos anteriormente o algoritmo Apriori.
Este algoritmo pode ser muito custoso computacionalmente em função da geração sucessiva de muitos conjuntos candidatos. 
Resolve-se isso usando o algoritmo que segue.

<h2> Desta vez vamos estudar o algorito FP-GROWTH </h2>

In [1]:
#dataset a ser utilizado (simples pra facilitar)
dataset = [[1,3],[1,2,3],[1],[1,3,4],[3],[1,2],[1,2,3,4]]

<table style="width:100%,align:left">
    <caption> <h3> Dataset Compras (Transacional) </h3> </caption>
  <tr>
    <th>Trans.</th>
    <th>1</th> 
    <th>2</th>
    <th>3</th>
    <th>4</th>
    <th>5</th>  
    <th>6</th>
    <th>7</th>   
  </tr>
  <tr>
    <td>1</td>
    <td>S</td> 
    <td>N</td>
    <td>S</td>
    <td>N</td>
    <td>N</td>
    <td>N</td>
    <td>N</td> 
  </tr>
  <tr>
    <td>2</td>
    <td>S</td> 
    <td>S</td>
    <td>S</td>
    <td>N</td>
    <td>N</td>
    <td>N</td>
    <td>N</td>
  </tr>
  <tr>
    <td>3</td>
    <td>S</td> 
    <td>N</td>
    <td>N</td>
    <td>N</td>
    <td>N</td>
    <td>N</td>
    <td>N</td>
  </tr>
  <tr>
    <td>4</td>
    <td>S</td> 
    <td>N</td>
    <td>S</td>
    <td>S</td>
    <td>N</td>
    <td>N</td>
    <td>N</td>
  </tr>
  <tr>
    <td>5</td>
    <td>N</td> 
    <td>N</td>
    <td>S</td>
    <td>N</td>
    <td>N</td>
    <td>N</td>
    <td>N</td>
  </tr>
  <tr>
    <td>6</td>
    <td>S</td> 
    <td>S</td>
    <td>N</td>
    <td>N</td>
    <td>N</td>
    <td>N</td>
    <td>N</td>
  </tr>
  <tr>
    <td>7</td>
    <td>S</td> 
    <td>S</td>
    <td>S</td>
    <td>S</td>
    <td>N</td>
    <td>N</td>
    <td>N</td>
  </tr> 
</table>

In [2]:
#dados de entrada
TID = dataset # conjunto de transações
fmin = 3 # frequencia mínima admissível

#saída
#L : lista de itens frequentes


In [3]:
#
# Aproveitando código do anterior 
#vamos criar uma função para retirar os itens do proprio dataset. Desta forma evitamos inconsistências
#
def capturaItens(lista):
    itens =[]
    for i in lista:
        for j in i:
            if j not in itens: itens.append(j)
    itens.sort()
    return itens

#testando
capturaItens(dataset)

[1, 2, 3, 4]

In [4]:
#Vamos criar uma função auxiliar para verificar se um conjunto é subconjunto de outro 
def isSubset(a,b):
    res = True
    for i in a:
        cont = 0
        for j in i:
            for z in b:
                if len(z)<len(i): continue
                if j in z: 
                    cont+=1
                    break
        if cont != len(i):
            return False
    return True

#Testando
print(isSubset([[1],[1,3]],[[1,4,2],[3,1,4]]))
print(isSubset([[]],[[1]]))
print(isSubset([[1,2,3,4],[1,2,3]],[[1],[2]]))
print(isSubset([[1,2,3,4],[1,2,3]],[[1],[2],[3],[4]]))
print(isSubset([[1,2,3,4],[1,2,3]],[[1,2,3,4],[1]]))
print(isSubset([[1]],[[1,3],[1,2,3],[1],[1,3,4],[3],[1,2],[1,2,3,4]]))


True
True
False
False
True
True


In [5]:
#passo 1. criar o 1-itemset (C1) e calcular a frequencia
def geraFrequencias(TID):
    c1 = [[i] for i in capturaItens(TID)]
    freqs = []
    for i in c1:
        n=0
        for j in TID:
            if isSubset([i],[j]): n+=1
        freqs.append(n)
    return c1,freqs
c1,freqs = geraFrequencias(TID)

In [6]:
#passo 2 selecionar os itens cuja frequencia é maior que a mínima
def filtraSequencias(c1,freqs):
    for i in range(len(freqs)):
        if freqs[i] < fmin:
            del c1[i]
            del freqs[i]
    return c1,freqs
c1,freqs = filtraSequencias(c1,freqs)

In [7]:
#passo tres: ordenar C1 em ordem inversa
def ordena(lista,freqs):
    if len(lista) != len(freqs): return "ERRO"
    maior = 0
    for i in range(len(lista)):
        maior = i
        for j in range(i+1,len(lista)):
            if freqs[j]>freqs[maior]:
                maior = j
        freqs[i],freqs[maior]=freqs[maior],freqs[i]
        lista[i],lista[maior]=lista[maior],lista[i]    
    return lista,freqs

lista,freqs = ordena(c1,freqs)
print(lista,freqs)

[[1], [3], [2]] [6, 5, 3]


In [8]:
#tambem precisamos ordenar as transações pelos itens já eliminando os menos frequentes
def ordenaTransacoes(tid,c):
    tidn = []
    for t in tid:
        nl=[]
        for i in c:
            if i[0] in t:
                nl.append(i[0])
        tidn.append(nl)
    return tidn
print(TID)
nTID = ordenaTransacoes(TID,c1)
print(nTID)

[[1, 3], [1, 2, 3], [1], [1, 3, 4], [3], [1, 2], [1, 2, 3, 4]]
[[1, 3], [1, 3, 2], [1], [1, 3], [3], [1, 2], [1, 3, 2]]


In [9]:
#passo quatro: criar uma estrutura hierarquica (Arvore)
class Noh:
    def __init__(self,rot="",pai=None,freq=1):
        self.rotulo = rot
        self.filhos = []
        self.frequencia = freq
        self.pai = pai
        
    def insereFilho(self,rot):
        for i in self.filhos:
            if rot == i.rotulo:
                i.frequencia+=1
                return i
        novoNoh = Noh(rot,self)
        self.filhos.append(novoNoh)
        return novoNoh
        
            
    def __str__(self):
        st = ""
        for i in self.filhos:
            st+=str(i.rotulo)+":"+str(i.frequencia)+","
        return "{}:{}->{}".format(self.rotulo,self.frequencia,st)
    def imprimeArvore(self,spc=""):
        print(spc,self)
        spc+="\t"
        for i in self.filhos:
            if isinstance(i,Noh): i.imprimeArvore(spc)

In [10]:
#
#testando a estrutura
#
raiz = Noh("A")
raiz.insereFilho("B")
raiz.insereFilho("C")
raiz.insereFilho("B")
raiz.imprimeArvore()
raiz.filhos[0].insereFilho("D")
raiz.filhos[0].insereFilho("E")
raiz.filhos[1].insereFilho("F")
raiz.filhos[1].insereFilho("G")

raiz.imprimeArvore()

 A:1->B:2,C:1,
	 B:2->
	 C:1->
 A:1->B:2,C:1,
	 B:2->D:1,E:1,
		 D:1->
		 E:1->
	 C:1->F:1,G:1,
		 F:1->
		 G:1->


Aparentemente tudo normal ...

In [11]:
#passo 5 : inserir elementos na árvore
def fpTree(nTID):
    raiz = Noh("{}",None,0)
    for t in nTID[:]:
        atual = raiz
        for i in t:
            atual = atual.insereFilho(i)
    return raiz
fpTree(nTID).imprimeArvore()        

 {}:0->1:6,3:1,
	 1:6->3:4,2:1,
		 3:4->2:2,
			 2:2->
		 2:1->
	 3:1->


Nossa arvore FP está pronta. Agora temos que fazer uma busca em profundidade para encontrarmos os CIFs

<figure>
  <img src="img/fptree.png" alt="Trulli" style="width:50%">
</figure>

In [12]:
#
# o segredo é fazer uma busca exaustiva para cada item
#
def calculaCifs(c1,raiz):
    cifs = []
    for it in reversed(c1):
        lista = []
        lista.append(raiz)
        while len(lista)>0:
            atual = lista.pop(0)
            if it[0] == atual.rotulo:
                cif = []
                at = atual
                while at!=None:
                    cif.append(at)
                    at = at.pai
                cifs.append(cif)
            if len(atual.filhos)>0:
                for i in atual.filhos:
                    lista.append(i)
    return cifs

In [13]:
#testando
def imprimeCifs(cifs):
    for cif in cifs:
        print("|",end="")
        for i in cif:
            print(i.rotulo,":(",i.frequencia,")",end="  ")
        print("|")

imprimeCifs(calculaCifs(c1,fpTree(nTID)))
    

|2 :( 1 )  1 :( 6 )  {} :( 0 )  |
|2 :( 2 )  3 :( 4 )  1 :( 6 )  {} :( 0 )  |
|3 :( 1 )  {} :( 0 )  |
|3 :( 4 )  1 :( 6 )  {} :( 0 )  |
|1 :( 6 )  {} :( 0 )  |


In [14]:
#eliminando os unitarios
def eliminaUnitarios(cifs):
    deletar = []
    for cif in cifs:
        if len(cif)<3:
            deletar.append(cif)
    for i in deletar:
        del cifs[cifs.index(i)]
    return cifs

#testando
conjuntosF = eliminaUnitarios(calculaCifs(c1,fpTree(nTID)))       

#imprimir
imprimeCifs(conjuntosF)    

|2 :( 1 )  1 :( 6 )  {} :( 0 )  |
|2 :( 2 )  3 :( 4 )  1 :( 6 )  {} :( 0 )  |
|3 :( 4 )  1 :( 6 )  {} :( 0 )  |


In [15]:
#propagando as frequencias
def propagaFrequencias(cofs):
    f = [k[0].frequencia for k in cofs]
    r = [k[0].rotulo for k in cofs]
    conjs =[]
    for i in range(len(cofs)):
        cj = []
        for j in range(1,len(cofs[i])):
            cj.append(cofs[i][j].rotulo)
        conjs.append(cj)
    return r,f,conjs

#testando
r,f,conjs = propagaFrequencias(conjuntosF)
for i in range(len(r)):
    print(r[i],f[i],conjs[i])    


2 1 [1, '{}']
2 2 [3, 1, '{}']
3 4 [1, '{}']


Funciona...

In [16]:
# mais uma função auxiliar - fundir dois ramos duplicados
def fundir(ca,fa,cb,fb):
    i=len(ca)-1
    j=len(cb)-1
    z = max(i,j)
    f = [0 for i in range(z+1)]
    c = [None for i in range(z+1)]
    while i>=0 or j>=0:
        if ca[i] == cb[j]:
            c[max(i,j)] = ca[i]
            f[max(i,j)] = fa+fb
            i-=1
            j-=1
            continue
        if j >= 0:
            c[j] = cb[j]
            f[j] = fb
            j-=1
            continue
        if i >= 0:
            c[i] = ca[i]
            f[i] = fa
            i-=1
            continue
    return c,f

#testando        
fundir([1, '{}'],1,[3, 1, '{}'],2)

([3, 1, '{}'], [2, 3, 3])

ok...

In [17]:
#Agora vamos unificar
def unificar(r,f,conjs):
    i = 0
    L = [] # saida final do algoritmo
    FF = [] # frequencias
    lim = len(r)
    while i<lim:
        fq = f[i]
        ri = r[i]
        cj = conjs[i]
        j = i+1
        while j<lim:
            if r[j] == r[i]:
                cj,fq = fundir(cj,fq,conjs[j],f[j])
                del conjs[j]
                del r[j]
                del f[j]
                lim-=1
            j+=1
        if isinstance(fq,int):fq = [fq,fq]
        L.append(cj)
        FF.append(fq)
        i+=1
    return r,L,FF

    

In [18]:
# ultimo detalhe : compor o conjunto final
def calculaL(r,l,ff,suporteMinimo=3):
    for i in range(len(l)):
        jmax = len(l[i])
        j=0
        while j<jmax:
            if ff[i][j] < suporteMinimo:
                del l[i][j]
                del ff[i][j]
                jmax-=1
            j+=1
    LL=[]
    for i,j in zip(r,l):
        lt=[i]
        for z in j:
            if z != "{}":
                lt.append(z)
        LL.append(lt)
    return LL

#execução final

a,b,c = unificar(r,f,conjs) 

print("Resultado Final ",calculaL(a,b,c))            


Resultado Final  [[2, 1], [3, 1]]


É isso ai. Agora podemos levantar as regras usando o mesmo método usado no apriori.

<h3> Boa Sorte com o FP-Growth </h3>